In [6]:
from transformers import BertForMaskedLM, BertTokenizer
from models.seqlab import DNABERT_SL
import json
import bertviz
import os
import torch


In [7]:
args = {
    "device": "cuda:0",
    "model-config": os.path.join("models", "config", "seqlab", "base.lin1.json"),
    "model-checkpoint": os.path.join("run", "sso01-adamw-lr5e-5-base.lin1-2w1boplw", "latest", "checkpoint.pth"),
    "test-config": os.path.join("training", "config", "seqlab", "ss-only.01.lr5e-5.json"),
    "log": os.path.join("prediction", "sso01-adamw-lr5e-5-base.lin1-2w1boplw")
}

device = args.get("device", "cpu") # specify device or use cpu otherwise.

model_config_path = args.get("model-config", False)
model_checkpoint = args.get("model-checkpoint", False)
test_config_path = args.get("test-config")
test_config = json.load(open(test_config_path, "r"))
test_file = test_config.get("test_data", False)

if not model_config_path:
    raise ValueError("model config not specified.")
if not os.path.exists(model_config_path):
    raise ValueError(f"model config not exists at {model_config_path}")
print(f"using model config at {model_config_path}")
    
if not model_checkpoint:
    raise ValueError("model checkpoint not specified.")
if not os.path.exists(model_checkpoint):
    raise ValueError(f"model checkpoint not exists at {model_checkpoint}")
print(f"found model checkpoint at {model_checkpoint}")

if not test_file:
    raise ValueError("test not specified.")
if not os.path.exists(test_file):
    raise ValueError(f"test file not exists at {test_file}")
print(f"found test data at {test_file}")

bert_for_masked_lm = BertForMaskedLM.from_pretrained(os.path.join("pretrained", "3-new-12w-0"))
model = DNABERT_SL(
    bert_for_masked_lm.bert, # bert, 
    json.load(open(model_config_path, "r")) # config
)

checkpoint = torch.load(model_checkpoint, map_location=device)
model.load_state_dict(checkpoint.get("model"))
model.eval()
model.to(device)

tokenizer = BertTokenizer.from_pretrained(os.path.join("pretrained", "3-new-12w-0"))

using model config at models\config\seqlab\base.lin1.json
found model checkpoint at run\sso01-adamw-lr5e-5-base.lin1-2w1boplw\latest\checkpoint.pth
found test data at workspace\seqlab-latest\gene_index.01_test_ss_all_pos.csv


In [14]:
input_str = "ACT CTT TTT TTT TTC TCC CCC CCT CTT TTT TTT TTT TTA TAT ATT TTT TTA TAC ACA CAG AGA GAA AAG AGG GGG GGT GTC TCT CTG TGG GGC GCT CTC TCT CTT TTA TAT ATC TCA CAC ACC CCT CTC TCT CTG TGA GAA AAT ATA TAG AGC GCA CAT ATG TGC GCC CCA CAT ATT TTC TCT CTC TCT CTT TTA TAG AGC GCC CCT CTC TCA CAT ATC TCT CTA TAA AAG AGA GAA AAA AAC ACA CAG AGA GAC ACT CTG TGT GTA TAG AGG GGG GGG GGA GAA AAG AGA GAG AGA GAA AAG AGA GAT ATC TCT CTA TAG AGA GAA AAT ATG TGT GTT TTG TGG GGT GTC TCA CAC ACC CCA CAA AAC ACC CCA CAA AAT ATG TGG GGC GCA CAG AGA GAC ACA CAT ATT TTA TAT ATC TCA CAG AGG GGT GTA TAA AAG AGA GAA AAA AAC ACC CCT CTG TGC GCC CCC CCA CAC ACC CCC CCA CAA AAA AAC ACT CTC TCT CTG TGG GGA GAC ACC CCC CCT CTA TAA AAT ATA TAT ATG TGC GCA CAA AAG AGT GTC TCT CTA TAA AAC ACT CTA TAG AGT GTT TTC TCC CCA CAT ATA TAA AAA AAA AAC ACC CCA CAA AAT ATC TCC CCC CCT CTG TGC GCT CTA TAA AAC ACC CCC CCC CCA CAC ACA CAA AAA AAC ACA CAG AGG GGT GTT TTG TGC GCC CCT CTA TAT ATA TAG AGC GCC CCT CTT TTA TAC ACA CAA AAG AGT GTG TGG GGC GCC CCT CTC TCC CCT CTT TTG TGG GGT GTT TTT TTT TTT TTA TAG AGT GTG TGG GGC GCA CAT ATA TAG AGT GTG TGC GCA CAG AGG GGT GTG TGA GAG AGC GCC CCA CAT ATA TAT ATA TAT ATT TTA TAT ATG TGC GCT CTT TTC TCT CTC TCA CAA AAT ATT TTT TTA TAT ATT TTT TTC TCC CCA CAT ATT TTG TGG GGT GTC TCA CAG AGA GAA AAA AAA AAC ACT CTG TGG GGT GTA TAA AAA AAA AAA AAA AAT ATT TTC TCC CCT CTT TTT TTT TTC TCA CAA AAT ATG TGG GGT GTA TAT ATA TAG AGT GTT TTT TTT TTC TCG CGA GAA AAT ATC TCT CTC TCA CAA AAC ACA CAG AGA GAG AGA GAC ACT CTT TTC TCT CTT TTT TTG TGG GGT GTC TCT CTT TTA TAG AGT GTG TGA GAT ATG TGA GAG AGT GTC TCT CTT TTC TCA CAT ATG TGG GGT GTA TAT ATT TTT TTT TTG TGG GGC GCT CTT TTG TGC GCA CAC ACG CGT GTA TAG AGT GTA TAA AAC ACA CAA AAT ATA TAA AAT ATC TCA CAA AAG AGT GTC TCA CAT ATT TTC TCA CAC ACC CCC CCA CAG AGT GTG TGC GCA CAA AAG AGA GAT ATT TTT TTT TTG TGG GGT GTG TGT GTC TCT CTT TTT TTA TAT ATT TTT TTT TTT TTA TAA AAC ACT CTA TAT ATT TTT TTG TGT GTA TAT ATT TTG TGT GTG TGC GCA CAT ATT TTA TAT ATC TCA CAC ACA CAT ATA TAG"
encoded = tokenizer(input_str)
input_ids = encoded.get("input_ids")
attention_mask = encoded.get("attention_mask")

arr_input_ids = [input_ids]
arr_input_ids = torch.tensor(arr_input_ids)
arr_attention_mask = [attention_mask]
arr_attention_mask = torch.tensor(arr_attention_mask)

y_pred, bert_output = model(
    arr_input_ids.to(device), 
    arr_attention_mask.to(device)
)
attention = bert_output.get("attentions")

In [19]:
from bertviz import model_view

tokens = tokenizer.convert_ids_to_tokens(input_ids)
model_view(attention, tokens)
